In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy.optimize import fsolve
import plotly.graph_objects as go
import plotly.express as px

## Loading Data

In [ ]:
def load_calibration_data(filename):
    col_names = ['theta_x', 'theta_y', 'qpd_x', 'qpd_y', 'qpd_sum']
    df = pd.read_csv(filename, names=col_names)
    return df

filename = '../data/2020-03-10_QPD-Tilt-Calibration_Test1.csv'
load_calibration_data(filename)

### Check Data Validity

In [ ]:
df.head()

## Surface Fitting
Fit 2D Functions of form $q_x=f(\theta_x, \theta_y)$ and $q_y=f(\theta_x, \theta_y)$

e.g. using 3rd order polynomials:

$q_x = a_1.\theta_x^3 + a_2.\theta_y^3 + a_3.\theta_x^2 + a_4.\theta_y^2 + a_5.\theta_x + a_6.\theta_y + a_7$

$q_y = b_1.\theta_x^3 + b_2.\theta_y^3 + b_3.\theta_x^2 + b_4.\theta_y^2 + b_5.\theta_x + b_6.\theta_y + b_7$

In [ ]:
def surface_fitting(x, y, z):
    X = x.flatten()
    Y = y.flatten()
    Z = z.flatten()

    A = np.array([X**3, Y**3, X**2, Y**2, X, Y, X*0+1]).T
    #print(A.shape)

    coeff, residuals, rank, s = np.linalg.lstsq(A, Z, rcond=None)
    
    #print('Fitting coefficients:', coeff)
    #print('Residuals:', residuals)
    #print('Rank:', rank)
    return coeff

In [ ]:
c = np.zeros([2, 7])
c[0, :] = surface_fitting(df['theta_x'].to_numpy(), df['theta_y'].to_numpy(), df['qpd_x'].to_numpy())
c[1, :] = surface_fitting(df['theta_x'].to_numpy(), df['theta_y'].to_numpy(), df['qpd_y'].to_numpy())
print(c)

### Plot the fitted surfaces

In [ ]:
def poly2Dreco(X, Y, c):
    return (c[0]*X**3 + c[1]*Y**3 + c[2]*X**2 + c[3]*Y**2 + c[4]*X + c[5]*Y + c[6])

def plot_fitted_surface(x_raw, y_raw, z_raw, c, plot_opts):
    
    grid_x, grid_y = define_grid(x_raw, y_raw)

    zfit = poly2Dreco(grid_x, grid_y, c)

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(x=x_raw, y=y_raw, z=z_raw, mode='markers'))


    fig.add_trace(go.Surface(z=zfit, x=grid_x, y=grid_y))
                    
    fig.update_layout(scene = plot_opts)

    fig.update_layout(
    autosize=False,
    width=800,
    height=800)
    
    fig.show()

def define_grid(x_raw, y_raw):

    x_min = x_raw.min()
    x_max = x_raw.max()
    y_min = y_raw.min()
    y_max = y_raw.max()

    x_step = 100 * 1j
    y_step = 100 * 1j

    #grid_x, grid_y = np.mgrid[-0.5:0.5:100j, -0.4:0.9:100j]
    grid_x, grid_y = np.mgrid[x_min:x_max:x_step, y_min:y_max:y_step]

    return grid_x, grid_y

Plot variation of QPD x-position with $\theta_x$ and $\theta_y$

In [ ]:
plot_opts = dict(
            xaxis_title='theta_x',
            yaxis_title='theta_y',
            zaxis_title='QPD x')
plot_fitted_surface(df['theta_x'], df['theta_y'], df['qpd_x'], c[0, :], plot_opts)

Plot variation of QPD x-position with $\theta_x$ and $\theta_y$

In [ ]:
plot_opts = dict(
            xaxis_title='theta_x',
            yaxis_title='theta_y',
            zaxis_title='QPD y')
plot_fitted_surface(df['theta_x'], df['theta_y'], df['qpd_y'], c[1, :], plot_opts)

# Calibration Procedure


## Numerical Calculation

In [ ]:
def calc_angles_from_qpd_values(c, qpd_pos, calculation_grid_dimensions):

    grid_tx, grid_ty = np.meshgrid(calculation_grid_dimensions['tx_vec'], calculation_grid_dimensions['ty_vec'])

    is_common_intersection = calc_common_intersection(grid_tx, grid_ty, c, qpd_pos)
    
    thetas = calc_angles_from_common_intersection(grid_tx, grid_ty, is_common_intersection)

    return thetas

def calc_common_intersection(grid_tx, grid_ty, c, qpd_pos):

    intersection_array = np.zeros((2,) + grid_tx.shape) # initialise

    for i in range(0, 2): # For qpd_x and qpd_y,
        # Calculate plane-surface intersection
        z_qpd = poly2Dreco(grid_tx, grid_ty, c[i, ]) - qpd_pos[i] # z_qpd_x == 0 where qpd_x == qpd_pos[0]
        is_intersection = np.abs(z_qpd - 0) < 0.01

        intersection_array[i,] = is_intersection

    # Calculate common intersection (i.e. find theta_x and theta_y position)
    is_common_intersection = np.all(intersection_array, axis=0)
    
    return is_common_intersection

def calc_angles_from_common_intersection(grid_tx, grid_ty, is_common_intersection):

    theta_x = np.mean(grid_tx[is_common_intersection])
    theta_y = np.mean(grid_ty[is_common_intersection])

    thetas = np.array([theta_x, theta_y])

    return thetas

In [ ]:
def verify_qpd_from_angles(thetas, c):

    x_qpd_chk = poly2Dreco(thetas[0], thetas[1], c[0, ])
    y_qpd_chk = poly2Dreco(thetas[0], thetas[1], c[1, ])

    print('verification qpd position: ' + str([x_qpd_chk, y_qpd_chk]))

In [ ]:
qpd_pos = np.array([0, 0.09])

calculation_grid_dimensions = dict(tx_vec=np.arange(-0.5, 0.5, 0.01),
                                   ty_vec=np.arange(-0.4, 0.9, 0.01))

thetas_numerical = calc_angles_from_qpd_values(c, qpd_pos, calculation_grid_dimensions)

print('input qpd position: ' + str(qpd_pos))
print('calculated angles are ' + str(thetas_numerical) + ' degrees')

verify_qpd_from_angles(thetas_numerical, c)

## Produce Calibration File
### Iterate through QPD positions

In [ ]:
def iterate_qpd_positions(grid_qx, grid_qy):
    tx_array = np.zeros_like(grid_qx)
    ty_array = np.zeros_like(grid_qx)
    
    for i, j in np.ndindex(grid_qx.shape):

        qpd_pos = np.array([grid_qx[i, j], grid_qy[i, j]])

        t = calc_angles_from_qpd_values(c, qpd_pos, calculation_grid_dimensions)
        
        tx_array[i, j] = t[0]
        ty_array[i, j] = t[1]
        
    print('done')

    return tx_array, ty_array

In [ ]:
# iterate through QPD positions
qx_vec = np.arange(-1, 1, 0.1)
qy_vec = np.arange(-1, 1, 0.1)
grid_qx, grid_qy = np.meshgrid(qx_vec, qy_vec)

tx_array, ty_array = iterate_qpd_positions(grid_qx, grid_qy)

In [ ]:
fig = px.scatter(x=grid_qx.flatten(), y=grid_qy.flatten(), color=tx_array.flatten())
fig.update_layout(xaxis_title='QPD x',
                  yaxis_title='QPD y')
fig.show()

Fit surface to calibration data:


In [ ]:
d = np.zeros([2, 7])
d[0, :] = surface_fitting(grid_qx.flatten(), grid_qy.flatten(), tx_array.flatten())
d[1, :] = surface_fitting(grid_qx.flatten(), grid_qy.flatten(), ty_array.flatten())
print(d)

Plot variation of $\theta_x$ with $q_x$ and $q_y$

In [ ]:
plot_opts = dict(
            xaxis_title='QPD x',
            yaxis_title='QPD y',
            zaxis_title='theta_x')
plot_fitted_surface(grid_qx.flatten(), grid_qy.flatten(), tx_array.flatten(), d[0, :], plot_opts)

Plot variation of $\theta_y$ with $q_x$ and $q_y$

In [ ]:
plot_opts = dict(
            xaxis_title='QPD x',
            yaxis_title='QPD y',
            zaxis_title='theta_y')
plot_fitted_surface(grid_qx.flatten(), grid_qy.flatten(), ty_array.flatten(), d[1, :], plot_opts)